# Polyp Segmentation for Computer Aided Gastrointestinal Disease Detection

## 1. Introduction

### 1.1 Colorectal Cancer

Colorectal cancer is the second most common cancer type among women and third most common among men. Abnormal regions, known as polyps, are potential indicators for colorectal cancer. Detecting these polyps early on can improve patient survival through timely treatment. Currently, colonoscopy is the best available method to detect and remove colonic polyps. Colonoscopy rarely misses polyps ≥10 mm, but **the miss rate increases signiﬁcantly in smaller sized polyps**. Hence, automatic detection of polyps can play a crucial role in prevention and survival from colorectal cancer.

### 1.2 Kvasir-SEG Dataset

Kvasir-SEG is an open-access dataset of 1000 gastrointestinal polyp images and manually annotated corresponding masks. Images have JPEG compression and resolutions ranging from 332x487 to 1920x1072 pixels. The images and their corresponding masks are stored in two separate folders with the same filename.

![Example Images from Kvasir-SEG](attachment:916dec30-b5c5-4216-b7b1-7ee9082863e9.png)

The figure presents examples of images from Kvasir-SEG. All 1000 images were manually annotated with the help of medical experts using the Labelbox software. The generated masks are 1-bit color depth images, where the polyp is white (representing the foreground), and the rest is black (background). The information about the bounding box is stored in JSON file format.

### 1.3 Dataset Details

* **Size**: 46.2 MB
* **Content**: 1000 polyp images with corresponding ground truth
* **Resolution Range**: 332 x 487 to 1920 x 1072
* **Storage**: Organized in two folders with identical filenames for images and masks
* **Bounding Box**: Stored in JSON file format

### 1.4 Download Kvasir-SEG

The dataset can be downloaded using the following link:
        [Download Kvasir-SEG](https://datasets.simula.no/kvasir/)


In [ ]:
# Importing necessary libraries

import os
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import random
import pathlib
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

In [ ]:
root = os.getcwd()
root

'/content'

In [ ]:
# Downloading Data

image_dir = root + '/drive/MyDrive/Colab Notebooks/images/'
mask_dir = root + '/drive/MyDrive/Colab Notebooks/masks/'

def load_data(image_dir, mask_dir):

    """
      Loads image and mask data from specified directories.

      Args:
          image_dir (str): Directory path containing images.
          mask_dir (str): Directory path containing corresponding masks.

      Returns:
          tuple: A tuple of two lists:
            - image_data: List of loaded image arrays.
            - mask_data: List of loaded mask arrays.
    """

    image_data = []
    mask_data = []
    images = os.listdir(image_dir)
    for i, image_name in enumerate(images):
        image = cv2.imread(image_dir+image_name, 1)
        image_data.append(np.array(image))
        mask = cv2.imread(mask_dir + image_name, 0)
        mask_data.append(np.array(mask))
    return image_data, mask_data

In [ ]:
image_data, mask_data = load_data(image_dir, mask_dir)

In [ ]:
# Preprocessing Data

def normalization(images, masks):

    """
    Normalize image and mask data.

    Args:
        images (list): List of image arrays.
        masks (list): List of mask arrays.

    Returns:
        tuple: Normalized images and masks.
            - x: Images normalized to range [0, 1].
            - y: Masks normalized to range [0, 1] and expanded to have a channel dimension.
    """

    x = np.array(images) / 255.
    y = np.expand_dims((np.array(masks)), 3) / 255.

    return x, y

def preprocessing(images, masks, desired_size, filter_func = None, filter_param_first = None, filter_param_second = None):

    """
    Preprocess image and mask data with resizing, filtering, and normalization.

    Args:
        images (list): List of image arrays.
        masks (list): List of mask arrays.
        desired_size (tuple): Desired output size for resizing (width, height).
        filter_func (function, optional): Filter function to apply on images (e.g., GaussianBlur, Canny).
        filter_param_first (any, optional): First parameter for the filter function.
        filter_param_second (any, optional): Second parameter for the filter function.

    Returns:
        tuple: Normalized images and masks after preprocessing.
    """

    filtered_images = []
    filtered_masks = []
    for img, mask in zip(images, masks):
        resized_image = cv2.resize(img, desired_size)
        resized_mask = cv2.resize(mask, desired_size)
        kernel = np.ones((5,5),np.uint8)
        resized_mask = cv2.morphologyEx(resized_mask, cv2.MORPH_OPEN, kernel)
        resized_mask = cv2.morphologyEx(resized_mask, cv2.MORPH_CLOSE, kernel)
        if filter_func:
          if filter_param_second is not None:
            filtered_image = filter_func(resized_image, filter_param_first, filter_param_second)
            filtered_mask =  resized_mask
          else:
            filtered_image = filter_func(resized_image, filter_param_first)
            filtered_mask =  resized_mask
        else:
          filtered_image = resized_image
          filtered_mask = resized_mask
        filtered_images.append(filtered_image)
        filtered_masks.append(filtered_mask)

    normalized_images, normalized_masks = normalization(filtered_images, filtered_masks)
    return normalized_images, normalized_masks

In [ ]:
# preprocessing Data by resizing to 64 x 64

prep_images, prep_masks = preprocessing(image_data, mask_data, (64, 64))

In [ ]:
def model_construction(input_shape, num_layers=4, initial_filters=16, filter_size=(3, 3), activation='relu',
    kernel_initializer='he_normal', dropout_positions=None, dropout_rate=0.3
):
    """
    Constructs a U-Net model with configurable parameters.

    Args:
        input_shape (tuple): Shape of the input image (height, width, channels).
        num_layers (int): Number of downsampling layers.
        initial_filters (int): Number of filters in the first layer.
        filter_size (tuple): Size of the convolutional filters (e.g., (3, 3)).
        activation (str): Activation function to use (e.g., 'relu').
        kernel_initializer (str): Kernel initializer for convolutional layers.
        dropout_positions (list, optional): Indices of layers after which dropout is applied.
        dropout_rate (float): Dropout rate for the dropout layers.

    Returns:
        model: Configured U-Net model.
    """
    inputs = Input(input_shape)
    filters = initial_filters

    # Encoder (downsampling path)
    encoder_layers = []
    x = inputs
    for i in range(num_layers - 1):
        x = Conv2D(filters, filter_size, activation=activation, kernel_initializer=kernel_initializer, padding='same')(x)
        x = Conv2D(filters, filter_size, activation=activation, kernel_initializer=kernel_initializer, padding='same')(x)
        encoder_layers.append(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        if dropout_positions and i in dropout_positions:
            x = Dropout(dropout_rate)(x)
        filters *= 2

    # Bottleneck
    filters
    x = Conv2D(filters, filter_size, activation=activation, kernel_initializer=kernel_initializer, padding='same')(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(filters, filter_size, activation=activation, kernel_initializer=kernel_initializer, padding='same')(x)

    l = num_layers
    len = num_layers * 2
    # Decoder (upsampling path)
    for i in reversed(range(num_layers - 1)):
        l = l + 1
        filters //= 2
        x = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(x)
        x = concatenate([x, encoder_layers[i]])
        if dropout_positions and l in dropout_positions:
            x = Dropout(dropout_rate)(x)
        x = Conv2D(filters, filter_size, activation=activation, kernel_initializer=kernel_initializer, padding='same')(x)
        x = Conv2D(filters, filter_size, activation=activation, kernel_initializer=kernel_initializer, padding='same')(x)


    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(prep_images, prep_masks, test_size =0.20, random_state = 0)

In [ ]:
# Define metrics


def iou_dice_coef(y_test, y_pred):
  intersection = np.logical_and(y_test, y_pred)
  union = np.logical_or(y_test, y_pred_thresholded)
  iou_score = np.sum(intersection) / np.sum(union)
  dice_score = (2 * np.sum(intersection)) / (np.sum(y_test) + np.sum(y_pred))
  print("IoU score: ", iou_score)
  print("Dice score: ", dice_score)


In [ ]:
model = model_construction(
    input_shape=(64, 64, 3),
    num_layers=7,
    initial_filters=8,
    filter_size=(3, 3),
    activation='relu',
    dropout_positions=[5],
    dropout_rate=0.3
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_19 (Conv2D)        │ (None, 64, 64, 8)      │            224 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 64, 64, 8)      │            584 │ conv2d_19[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 32, 32, 8)      │              0 │ conv2d_20[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_21 (Conv2D)        │ (None, 32, 32, 16)     │          1,168 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_22 (Conv2D)        │ (None, 32, 32, 16)     │          2,320 │ conv2d_21[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 16, 16, 16)     │              0 │ conv2d_22[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_23 (Conv2D)        │ (None, 16, 16, 32)     │          4,640 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_24 (Conv2D)        │ (None, 16, 16, 32)     │          9,248 │ conv2d_23[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 8, 8, 32)       │              0 │ conv2d_24[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_25 (Conv2D)        │ (None, 8, 8, 64)       │         18,496 │ max_pooling2d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_26 (Conv2D)        │ (None, 8, 8, 64)       │         36,928 │ conv2d_25[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_7           │ (None, 4, 4, 64)       │              0 │ conv2d_26[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_27 (Conv2D)        │ (None, 4, 4, 128)      │         73,856 │ max_pooling2d_7[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_28 (Conv2D)        │ (None, 4, 4, 128)      │        147,584 │ conv2d_27[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_8           │ (None, 2, 2, 128)      │              0 │ conv2d_28[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_29 (Conv2D)   

 Total params: 7,779,385 (29.68 MB)

 Trainable params: 7,779,385 (29.68 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Fit the model with epochs=25 and batch_size=32

In [ ]:
history2 = model.fit(X_train, y_train, verbose=1, epochs=50, batch_size=16, shuffle=False)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7653 - loss: 0.5494
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8416 - loss: 0.3710
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8416 - loss: 0.3485
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8416 - loss: 0.3345
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8416 - loss: 0.3263
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8416 - loss: 0.3232
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8416 - loss: 0.3181
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8416 - loss: 0.3122
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8416 - loss: 0.3016
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8426 - loss: 0.2912
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8673 - loss: 0.2812
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy

In [ ]:
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

iou_dice_coef(y_test, y_pred_thresholded)

7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 301ms/step
IoU score:  0.43153751600185974
Dice score:  0.6144778920527628


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history2 = model.fit(X_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.9818 - loss: 0.0215
Epoch 2/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9819 - loss: 0.0216
Epoch 3/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9828 - loss: 0.0189
Epoch 4/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9839 - loss: 0.0156
Epoch 5/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9844 - loss: 0.0139
Epoch 6/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9846 - loss: 0.0128
Epoch 7/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9850 - loss: 0.0117
Epoch 8/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9852 - loss: 0.0114
Epoch 9/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9846 - loss: 0.0121
Epoch 10/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9847 - loss: 0.0117
Epoch 11/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9849 - loss: 0.0118
Epoch 12/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accurac

In [ ]:
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

iou_dice_coef(y_test, y_pred_thresholded)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step
IoU score:  0.47157947386597754
Dice score:  0.6520898075233872


In [ ]:
model = model_construction(
    input_shape=(64, 64, 3),
    num_layers=7,
    initial_filters=16,
    filter_size=(3, 3),
    activation='relu',
    dropout_positions=[6, 7, 8],
    dropout_rate=0.3
)
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12            │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_250 (Conv2D)       │ (None, 64, 64, 16)     │            448 │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_251 (Conv2D)       │ (None, 64, 64, 16)     │          2,320 │ conv2d_250[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_73          │ (None, 32, 32, 16)     │              0 │ conv2d_251[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_252 (Conv2D)       │ (None, 32, 32, 32)     │          4,640 │ max_pooling2d_73[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_253 (Conv2D)       │ (None, 32, 32, 32)     │          9,248 │ conv2d_252[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_74          │ (None, 16, 16, 32)     │              0 │ conv2d_253[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_254 (Conv2D)       │ (None, 16, 16, 64)     │         18,496 │ max_pooling2d_74[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_255 (Conv2D)       │ (None, 16, 16, 64)     │         36,928 │ conv2d_254[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_75          │ (None, 8, 8, 64)       │              0 │ conv2d_255[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_256 (Conv2D)       │ (None, 8, 8, 128)      │         73,856 │ max_pooling2d_75[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_257 (Conv2D)       │ (None, 8, 8, 128)      │        147,584 │ conv2d_256[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_76          │ (None, 4, 4, 128)      │              0 │ conv2d_257[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_258 (Conv2D)       │ (None, 4, 4, 256)      │        295,168 │ max_pooling2d_76[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_259 (Conv2D)       │ (None, 4, 4, 256)      │        590,080 │ conv2d_258[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_77          │ (None, 2, 2, 256)      │              0 │ conv2d_259[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_260 (Conv2D)  

 Total params: 31,110,001 (118.68 MB)

 Trainable params: 31,110,001 (118.68 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history2 = model.fit(X_train, y_train, epochs=32, batch_size=32)

Epoch 1/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 46ms/step - accuracy: 0.9346 - loss: 0.1576
Epoch 2/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.9529 - loss: 0.0967
Epoch 3/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9643 - loss: 0.0684
Epoch 4/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9665 - loss: 0.0621
Epoch 5/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9695 - loss: 0.0551
Epoch 6/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9700 - loss: 0.0535
Epoch 7/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9714 - loss: 0.0499
Epoch 8/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9725 - loss: 0.0472
Epoch 9/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9737 - loss: 0.0437
Epoch 10/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9727 - loss: 0.0463
Epoch 11/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9714 - loss: 0.0499
Epoch 12/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy

In [ ]:
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

iou_dice_coef(y_test, y_pred_thresholded)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step
IoU score:  0.5279398563734291
Dice score:  0.7031974163589249


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history2 = model.fit(X_train, y_train, epochs=32, batch_size=32)

In [ ]:
prep_images, prep_masks = preprocessing(image_data, mask_data, (256, 256))
X_train, X_test, y_train, y_test = train_test_split(prep_images, prep_masks, test_size =0.20, random_state = 0)

In [ ]:
model = model_construction(
    input_shape=(256, 256, 3),
    num_layers=7,
    initial_filters=16,
    filter_size=(3, 3),
    activation='relu',
    dropout_positions=[6, 7, 8],
    dropout_rate=0.3
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 16)   │            448 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 16)   │          2,320 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 16)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 32)   │          4,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 32)   │          9,248 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 32)     │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 64)     │         36,928 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 64)     │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 128)    │         73,856 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 32, 32, 128)    │        147,584 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 16, 16, 128)    │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 16, 16, 256)    │        295,168 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 16, 16, 256)    │        590,080 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 8, 8, 256)      │              0 │ conv2d_9[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 8, 8, 512)      │      1,180,160 │ max_pooling2d_4[0][0]  │
├──────────────────────

 Total params: 31,110,001 (118.68 MB)

 Trainable params: 31,110,001 (118.68 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history2 = model.fit(X_train, y_train, epochs=32, batch_size=32)

Epoch 1/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 52s 298ms/step - accuracy: 0.7862 - loss: 2.7972
Epoch 2/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 294ms/step - accuracy: 0.8457 - loss: 0.5406
Epoch 3/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 295ms/step - accuracy: 0.8351 - loss: 0.5107
Epoch 4/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - accuracy: 0.8434 - loss: 0.4321
Epoch 5/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 298ms/step - accuracy: 0.8400 - loss: 0.3661
Epoch 6/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 302ms/step - accuracy: 0.8481 - loss: 0.3415
Epoch 7/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 305ms/step - accuracy: 0.8468 - loss: 0.3328
Epoch 8/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 304ms/step - accuracy: 0.8518 - loss: 0.3256
Epoch 9/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 306ms/step - accuracy: 0.8534 - loss: 0.3153
Epoch 10/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 306ms/step - accuracy: 0.8553 - loss: 0.3215
Epoch 11/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 305ms/step - accuracy: 0.8638 - loss: 0.3075
Epoch 12/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 303

In [ ]:
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

iou_dice_coef(y_test, y_pred_thresholded)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step
IoU score:  0.5607071537148137
Dice score:  0.7253175490018998


In [ ]:
prep_images, prep_masks = preprocessing(image_data, mask_data, (256, 256), cv2.GaussianBlur, (5,5),0)
X_train, X_test, y_train, y_test = train_test_split(prep_images, prep_masks, test_size =0.20, random_state = 0)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history2 = model.fit(X_train, y_train, epochs=32, batch_size=32)

Epoch 1/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 302ms/step - accuracy: 0.8996 - loss: 0.2652
Epoch 2/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 298ms/step - accuracy: 0.9274 - loss: 0.1705
Epoch 3/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 299ms/step - accuracy: 0.9392 - loss: 0.1459
Epoch 4/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 302ms/step - accuracy: 0.9484 - loss: 0.1166
Epoch 5/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 306ms/step - accuracy: 0.9546 - loss: 0.1004
Epoch 6/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 305ms/step - accuracy: 0.9611 - loss: 0.0835
Epoch 7/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 305ms/step - accuracy: 0.9637 - loss: 0.0749
Epoch 8/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 306ms/step - accuracy: 0.9652 - loss: 0.0715
Epoch 9/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 307ms/step - accuracy: 0.9693 - loss: 0.0593
Epoch 10/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 304ms/step - accuracy: 0.9709 - loss: 0.0559
Epoch 11/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 304ms/step - accuracy: 0.9719 - loss: 0.0518
Epoch 12/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 3

In [ ]:
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

iou_dice_coef(y_test, y_pred_thresholded)

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step
IoU score:  0.5909817073092573
Dice score:  0.7505199939755918


## Summary
To enhance the accuracy of the polyp segmentation model, several techniques were employed throughout the notebook. The data preprocessing process was carefully refined, including normalization, resizing to multiple dimensions (e.g., 64x64 and 256x256), and applying techniques like Gaussian Blur to improve image quality. The model architecture was iteratively optimized by experimenting with the number of convolutional layers, filter sizes, and dropout layers to prevent overfitting. Different configurations for hyperparameters such as the number of epochs, batch sizes, and the learning rate were tested to achieve better performance. Metrics like the Intersection over Union (IoU) and Dice Coefficient were calculated to evaluate segmentation quality, and adjustments were made to improve these scores. Additionally, various dropout positions and rates were explored to ensure the model generalizes well, and preprocessing was tailored to the input size to handle different resolutions effectively. These combined efforts contributed to increasing the model's accuracy to IoU 59% and Dice score 75%


## Additional Comments

Initially, a comprehensive automated model was designed to handle multiple tasks and integrate all functionalities into a unified function. It wouldve allow to change easly different parameteres and test.

In [ ]:
def run_experiments(preprocessing_params, model_construct_params, model_run_params, images, masks):
    """
    Runs experiments with various preprocessing and model parameters.

    Returns:
        pd.DataFrame: Results of all experiments with metrics and parameters.
    """
    results = []

    for prep_index, prep_params in enumerate(preprocessing_params):
        # Preprocess data
        x, y = preprocessing(images, masks, prep_params["size"])

        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size =0.20, random_state = 0)
        print(x.shape)
        print(y.shape)

        for model_index, model_param in enumerate(model_params):
            # Construct model
            model = model_construction(model_param["input_shape"],
                                       model_param["num_layers"],
                                       model_param["initial_filters"],
                                       model_param["filter_size"],
                                       model_param["activation"],
                                       model_param["dropout_positions"],
                                       model_param["dropout_rate"])

            for model_run_index, model_run_param in enumerate(model_run_params):
            # Compile model
                model.compile(optimizer = model_run_param["optimizer"],
                              loss = model_run_param["loss"],
                              metrics=["accuracy", iou_coef, dice_coef])

                history = model.fit(X_train, y_train, epochs=model_run_param["epochs"], batch_size = model_run_param["batch_size"], verbose=0)

                y_pred =  model.predict(X_test, verbose=0)
                # Evaluate model
                scores = model.evaluate(y_test, y_pred, verbose=0)

                model_evaluation_visualization(X_test, y_test, y_pred)

                # Log results
                results.append({
                  "Preprocessing Index": prep_index + 1,
                  "Preprocessing Parameters": prep_params,
                  "Model Index": model_index + 1,
                  "Model Parameters": model_param_set,
                  "Accuracy": scores[1],
                  "IoU": iou_score,
                  "Dice Coefficient": scores[2]
                })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Preprocessing parameters
preprocessing_params = [
  {
        "size": (256, 256),
  }
]

# Model construction parameters
model_params = [
  {
        "input_shape": (256, 256, 3),
        "num_layers": 4,
        "initial_filters": 8,
        "filter_size": (3, 3),
        "activation": "relu",
        "dropout_positions": [2],
        "dropout_rate": 0.3
},
  {
        "input_shape": (256, 256, 3),
        "num_layers": 5,
        "initial_filters": 16,
        "filter_size": (3, 3),
        "activation": "relu",
        "dropout_positions": [3, 5],
        "dropout_rate": 0.4
  }
]

model_run_params = [
  {
        "optimizer": "adam",
        "loss": "binary_crossentropy",
        "batch_size": 8,
        "epochs": 25
  }
]


results_df = run_experiments(preprocessing_params, model_params, model_run_params, image_data, mask_data)


Additionaly, the model was tested on reduced image sizes (64x64) to minimize memory and processing requirements. Although 64x64 was computationally manageable, better segmentation results were observed with higher resolution images (256x256). Similarly, while data augmentation techniques were considered to enhance the training dataset and potentially improve accuracy, they were not fully implemented due to their significant demand on computational resources.